# PROJECT
Project creating probabilities of households picking each choice of the 500 alternatives from body type, fuel type and age

In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
'''
these files will be our input and contain household data from NHTS, vehicle cost data
and EPA data on body type, fuel type and age

'''
EPA = pd.read_csv('combined epa and veh cost.csv')
#epa_and_vehiclecost.set_index("CHOICE4",inplace=True,drop=True)
NHTS = pd.read_csv('NHTS2017.csv')
NHTS.set_index("HOUSEID",inplace=True,drop=True)
veh910  = pd.read_csv('vehtype910.csv', usecols=[0,1])
veh910.dropna(subset=["variable"],inplace=True)
veh910.set_index("variable",inplace=True,drop=True)

In [ ]:
'''
These variables are defined in the alogit file and will be making use of codes from the NHTS codebook and table
   
    INC025   = ifin(HHFAMINC,1,3)      
    INC2550  = ifin(HHFAMINC,4,5)      
    INC50100 = ifin(HHFAMINC,6,7)   
    INC100150= ifin(HHFAMINC,8,9)  
    INC150P  = ifge(HHFAMINC,10)  
    INC_MISS = iflt(HHFAMINC,0)
    RURALa(a) = a * RURAL
    ONEVEH = ifeq(HHVEHCNT,1)
    VHLTDR = ifgt(HHVEHCNT,1) * iflt(HHVEHCNT,PERSONS16P)
    VHEQDR = ifgt(HHVEHCNT,1) * ifeq(HHVEHCNT,PERSONS16P)
    VHGTDR = ifgt(HHVEHCNT,1) * ifgt(HHVEHCNT,PERSONS16P)
    NUM_CHLD3 = min(YOUNGCHILD + CHILDREN, 3)
    DSTWK = min(DSTTOWKAVG, 100)
    SFO = ifeq(HH_CBSA, 41860) + ifeq(HH_CBSA, 47900)
    SAN = ifeq(HH_CBSA, 41740)
    ATL = ifeq(HH_CBSA, 12060) 
    SEA = ifeq(HH_CBSA, 42660) 
    DET = ifeq(HH_CBSA, 19820) 
    MSP = ifeq(HH_CBSA, 33460) 
    DCA = ifeq(HH_CBSA, 47900) 
    OREG = ifeq(HHSTFIPS, 41) 
    OHIO = ifeq(HHSTFIPS, 39)  
    SMSA = ifeq(HH_CBSA, 0) * ifeq(RURAL, 0)
    OWN_VAN = ifge(VAN_COUNT, 1)
    OWN_SUV = ifge(SUV_COUNT, 1)
    OWN_PU  = ifge(PU_COUNT, 1)
    OWN_MC  = ifge(MC_COUNT, 1)
    OWN_HYB = ifge(HYB_COUNT, 1)
    OWN_PEV = ifge(PEV_COUNT, 1)
    OWN_BEV = ifge(BEV_COUNT, 1)
    OWN_EV = ifge(PEV_COUNT + BEV_COUNT, 1)
    DEN1 = ifeq(HBRESDN, 50)
    DEN2 = ifeq(HBRESDN, 300)
    DEN3 = ifeq(HBRESDN, 750)
    DEN4 = ifeq(HBRESDN, 1500)
    DEN5 = ifeq(HBRESDN, 3000)
    DEN6 = ifeq(HBRESDN, 7000)
    DEN7 = ifeq(HBRESDN, 17000)
    DEN8 = ifeq(HBRESDN, 30000)
    OWN_VAN = ifge(VAN_COUNT, 1)
    OWN_SUV = ifge(SUV_COUNT, 1)
    OWN_PU  = ifge(PU_COUNT, 1)
    OWN_MC  = ifge(MC_COUNT, 1)
    OWN_HYB = ifge(HYB_COUNT, 1)
    OWN_PEV = ifge(PEV_COUNT, 1)
    OWN_BEV = ifge(BEV_COUNT, 1)
    OWN_EV = ifge(PEV_COUNT + BEV_COUNT, 1)
    DEN1 = ifeq(HBRESDN, 50)
    DEN2 = ifeq(HBRESDN, 300)
    DEN3 = ifeq(HBRESDN, 750)
    DEN4 = ifeq(HBRESDN, 1500)
    DEN5 = ifeq(HBRESDN, 3000)
    DEN6 = ifeq(HBRESDN, 7000)
    DEN7 = ifeq(HBRESDN, 17000)
    DEN8 = ifeq(HBRESDN, 30000)
    
    RURAL = ifeq(URBRUR,2)
    LN_NMODELS(i) = log(1+NMODELS(i))
    LN_CHG_PC = log(1+CHG_PC)
    MSA1P   = ifin(MSASIZE,4,5)
    DSTWKT = min(DSTTOWKSUM, 100*WORKERS)
    RENT = ifeq(HOMEOWN, 1)
    SMSA = ifeq(HH_CBSA, 0) * ifeq(RURAL, 0)
'''

In [3]:
#starting with income which is the first 6 variables above
#defining the columns that give the conditions for HHFAMINC
NHTS['INC025'] = 0
NHTS['INC025'] = np.where((NHTS['HHFAMINC']>=1) & (NHTS['HHFAMINC']<=3), 1, 0)
NHTS['INC2550'] = 0
NHTS['INC2550'] = np.where((NHTS['HHFAMINC']>=4) & (NHTS['HHFAMINC']<=5), 1, 0)
NHTS['INC50100'] = 0
NHTS['INC50100'] = np.where((NHTS['HHFAMINC']>=6) & (NHTS['HHFAMINC']<=7), 1, 0)
NHTS['INC100150'] = 0
NHTS['INC100150'] = np.where((NHTS['HHFAMINC']>=8) & (NHTS['HHFAMINC']<=9), 1, 0)
NHTS['INC150P'] = 0
NHTS['INC150P'] = np.where((NHTS['HHFAMINC']>=10), 1, 0)
NHTS['INC_MISS'] = 0
NHTS['INC_MISS'] = np.where((NHTS['HHFAMINC']<=0), 1, 0)
NHTS['ONEVEH'] = 0
NHTS['ONEVEH'] = np.where((NHTS['HHVEHCNT'] ==1), 1, 0)
NHTS['VHLTDR'] = 0
NHTS['VHLTDR'] = np.where((NHTS['HHVEHCNT']>1) & (NHTS['HHVEHCNT'] < NHTS['PERSONS16P']), 1, 0)
NHTS['VHEQDR'] = 0
NHTS['VHEQDR'] = np.where((NHTS['HHVEHCNT']>1) & (NHTS['HHVEHCNT'] == NHTS['PERSONS16P']), 1, 0)
NHTS['VHGTDR'] = 0
NHTS['VHGTDR'] = np.where((NHTS['HHVEHCNT']>1) & (NHTS['HHVEHCNT'] > NHTS['PERSONS16P']), 1, 0)
NHTS['NUM_CHILD3'] = 0
NHTS['NUM_CHILD3'] = np.where((min(NHTS['YOUNGCHILD'] + NHTS['CHILDREN']) == 3), 1, 0)
NHTS['DSTWK'] = 0
NHTS['DSTWK'] = np.where(min(NHTS['DISTTOWK_AVG'] == 100), 1, 0)
NHTS['DSTWKT'] = 0
NHTS['DSTWKT'] = np.where(min(NHTS['DISTTOWK_SUM'] == 100*NHTS['WORKERS']), 1, 0)
NHTS['SFO'] = 0
NHTS['SFO'] = np.where((NHTS['HH_CBSA'] == 41860) & (NHTS['HH_CBSA'] == 47900), 1, 0)
NHTS['SAN'] = 0
NHTS['SAN'] = np.where((NHTS['HH_CBSA'] == 41740), 1, 0)
NHTS['ATL'] = 0
NHTS['ATL'] = np.where((NHTS['HH_CBSA'] == 12060), 1, 0)
NHTS['SEA'] = 0
NHTS['SEA'] = np.where((NHTS['HH_CBSA'] == 42660), 1, 0)
NHTS['DET'] = 0
NHTS['DET'] = np.where((NHTS['HH_CBSA'] == 19820), 1, 0)
NHTS['MSP'] = 0
NHTS['MSP'] = np.where((NHTS['HH_CBSA'] == 33460), 1, 0)
NHTS['DCA'] = 0
NHTS['DCA'] = np.where((NHTS['HH_CBSA'] == 47900), 1, 0)
NHTS['OREG'] = 0
NHTS['OREG'] = np.where((NHTS['HHSTFIPS'] == 41), 1, 0)
NHTS['OHIO'] = 0
NHTS['OHIO'] = np.where((NHTS['HHSTFIPS'] == 39), 1, 0)
NHTS['OWN_VAN'] = 0
NHTS['OWN_VAN'] = np.where((NHTS['VAN_COUNT'] >= 1), 1, 0)
NHTS['OWN_PU'] = 0
NHTS['OWN_PU'] = np.where((NHTS['PU_COUNT'] >= 1), 1, 0)
NHTS['OWN_SUV'] = 0
NHTS['OWN_SUV'] = np.where((NHTS['SUV_COUNT'] >= 1), 1, 0)
NHTS['OWN_MC'] = 0
NHTS['OWN_MC'] = np.where((NHTS['MC_COUNT'] >= 1), 1, 0)
NHTS['OWN_HYB'] = 0
NHTS['OWN_HYB'] = np.where((NHTS['HYB_COUNT'] >= 1), 1, 0)
NHTS['OWN_PEV'] = 0
NHTS['OWN_PEV'] = np.where((NHTS['PEV_COUNT'] >= 1), 1, 0)
NHTS['OWN_BEV'] = 0
NHTS['OWN_BEV'] = np.where((NHTS['BEV_COUNT'] >= 1), 1, 0)
NHTS['OWN_EV'] = 0
NHTS['OWN_EV'] = np.where((NHTS['PEV_COUNT'] + NHTS['BEV_COUNT'] >= 1), 1, 0)
NHTS['DEN1'] = 0
NHTS['DEN1'] = np.where((NHTS['HBRESDN'] == 50), 1, 0)
NHTS['DEN2'] = 0
NHTS['DEN2'] = np.where((NHTS['HBRESDN'] == 300), 1, 0)
NHTS['DEN3'] = 0
NHTS['DEN3'] = np.where((NHTS['HBRESDN'] == 750), 1, 0)
NHTS['DEN4'] = 0
NHTS['DEN4'] = np.where((NHTS['HBRESDN'] == 1500), 1, 0)
NHTS['DEN5'] = 0
NHTS['DEN5'] = np.where((NHTS['HBRESDN'] == 3000), 1, 0)
NHTS['DEN6'] = 0
NHTS['DEN6'] = np.where((NHTS['HBRESDN'] == 7000), 1, 0)
NHTS['DEN7'] = 0
NHTS['DE71'] = np.where((NHTS['HBRESDN'] == 17000), 1, 0)
NHTS['DEN8'] = 0
NHTS['DEN8'] = np.where((NHTS['HBRESDN'] == 30000), 1, 0)
NHTS['ALL_VAN'] = 0
NHTS['ALL_VAN'] = np.where((NHTS['VAN_COUNT'] == NHTS['HHVEHCNT']), 1, 0)
NHTS['ALL_SUV'] = 0
NHTS['ALL_SUV'] = np.where((NHTS['SUV_COUNT'] == NHTS['HHVEHCNT']), 1, 0)
NHTS['ALL_PU'] = 0
NHTS['ALL_PU'] = np.where((NHTS['PU_COUNT'] == NHTS['HHVEHCNT']), 1, 0)
NHTS['ALL_MC'] = 0
NHTS['ALL_MC'] = np.where((NHTS['MC_COUNT'] == NHTS['HHVEHCNT']), 1, 0)
NHTS['ALL_HYB'] = 0
NHTS['ALL_HYB'] = np.where((NHTS['HYB_COUNT'] == NHTS['HHVEHCNT']), 1, 0)
NHTS['ALL_PEV'] = 0
NHTS['ALL_PEV'] = np.where((NHTS['PEV_COUNT'] == NHTS['HHVEHCNT']), 1, 0)
NHTS['ALL_BEV'] = 0
NHTS['ALL_BEV'] = np.where((NHTS['BEV_COUNT'] == NHTS['HHVEHCNT']), 1, 0)
NHTS['RURAL'] = 0
NHTS['RURAL'] = np.where((NHTS['URBRUR'] == 2), 1, 0)
NHTS['MSA1P'] = 0
NHTS['MSA1P'] = np.where((NHTS['MSASIZE']>=4) & (NHTS['MSASIZE']<=5), 1, 0)
NHTS['RENT'] = 0
NHTS['RENT'] = np.where((NHTS['HOMEOWN'] ==1), 1, 0)



In [ ]:
#Vehicle type: car:1, van:2, suv:3, pickup:4, motorcycle:5
#Age: 1 - 20 years
#Fuel type: gas:1, diesel:2, hybrid:3, pev:4, bev:5


veh_type = ["1","2","3","4","5"]
fuel_type = ["1","2","3","4","5"]
car_age = ["01","02","03","04","05","06","07","08","09","10","11","12","13","14","15","16","17","18","19","20"]
for veh in veh_type:
    for fuel in fuel_type:
        for age in car_age:
            temp = veh + fuel + str(age)
            print(f"Choice: {temp}")

In [4]:
#decided to try another way and call the function from EPA data instead
#Here I am defining each column for each option like above 
EPA['car'] = 0
EPA['car'] = np.where((EPA['VEHTYPE2_TXT']== 'Car'), 1, 0)
EPA['van'] = 0
EPA['van'] = np.where((EPA['VEHTYPE2_TXT']== 'Van'), 1, 0)
EPA['suv'] = 0
EPA['suv'] = np.where((EPA['VEHTYPE2_TXT']== 'SUV'), 1, 0)
EPA['pu'] = 0
EPA['pu'] = np.where((EPA['VEHTYPE2_TXT']== 'Pickup'), 1, 0)
EPA['mc'] = 0
EPA['mc'] = np.where((EPA['VEHTYPE2_TXT']== 'Motorcycle'), 1, 0)
EPA['age_2'] = 0
EPA['age_2'] = np.where((EPA['VEHAGE_C20']== '2'), 1, 0)
EPA['age_3'] = 0
EPA['age_3'] = np.where((EPA['VEHAGE_C20']== '3'), 1, 0)
EPA['age_4'] = 0
EPA['age_4'] = np.where((EPA['VEHAGE_C20']== '4'), 1, 0)
EPA['age_5'] = 0
EPA['age_5'] = np.where((EPA['VEHAGE_C20']== '5'), 1, 0)
EPA['age_6'] = 0
EPA['age_6'] = np.where((EPA['VEHAGE_C20']== '6'), 1, 0)
EPA['age_7'] = 0
EPA['age_7'] = np.where((EPA['VEHAGE_C20']== '7'), 1, 0)
EPA['age_8'] = 0
EPA['age_8'] = np.where((EPA['VEHAGE_C20']== '8'), 1, 0)
EPA['age_9'] = 0
EPA['age_9'] = np.where((EPA['VEHAGE_C20']== '9'), 1, 0)
EPA['age_10'] = 0
EPA['age_10'] = np.where((EPA['VEHAGE_C20']== '10'), 1, 0)
EPA['age_11'] = 0
EPA['age_11'] = np.where((EPA['VEHAGE_C20']== '11'), 1, 0)
EPA['age_12'] = 0
EPA['age_12'] = np.where((EPA['VEHAGE_C20']== '12'), 1, 0)
EPA['age_13'] = 0
EPA['age_13'] = np.where((EPA['VEHAGE_C20']== '13'), 1, 0)
EPA['age_14'] = 0
EPA['age_14'] = np.where((EPA['VEHAGE_C20']== '14'), 1, 0)
EPA['age_15'] = 0
EPA['age_15'] = np.where((EPA['VEHAGE_C20']== '15'), 1, 0)
EPA['age_16'] = 0
EPA['age_16'] = np.where((EPA['VEHAGE_C20']== '16'), 1, 0)
EPA['age_17'] = 0
EPA['age_17'] = np.where((EPA['VEHAGE_C20']== '17'), 1, 0)
EPA['age_18'] = 0
EPA['age_18'] = np.where((EPA['VEHAGE_C20']== '18'), 1, 0)
EPA['age_19'] = 0
EPA['age_19'] = np.where((EPA['VEHAGE_C20']== '19'), 1, 0)
EPA['age_20'] = 0
EPA['age_20'] = np.where((EPA['VEHAGE_C20']== '20'), 1, 0)
EPA['gas'] = 0
EPA['gas'] = np.where((EPA['FUELTYPE2_TXT']== 'Gas'), 1, 0)
EPA['diesel'] = 0
EPA['diesel'] = np.where((EPA['FUELTYPE2_TXT']== 'Diesel'), 1, 0)
EPA['hybrid'] = 0
EPA['hybrid'] = np.where((EPA['FUELTYPE2_TXT']== 'Hybrid'), 1, 0)
EPA['pev'] = 0
EPA['pev'] = np.where((EPA['FUELTYPE2_TXT']== 'PEV'), 1, 0)
EPA['bev'] = 0
EPA['bev'] = np.where((EPA['FUELTYPE2_TXT']== 'BEV'), 1, 0)

In [ ]:
#First method
#Here I am making a for loop that will go througheach body type and fuel type within the if statement
#I got stuck here because I don't know how to then set each result to their respective alternative
#i.e 1101, 1102,....
df = pd.DataFrame()
ages = [0, 0.171916334, 0.112524355, 0.010417899, -0.08639802, -0.330472882, -0.238477625, -0.38162028, -0.153686593, -0.186548952, -0.08313971, -0.056656758, -0.284882802, -0.396864966, -0.568987962, -0.727763108, -0.850963122, -0.978545665, -1.302052026, 0.488821831]
for c in EPA['car']:
    for i in range (1101, 1520):
        if c > 0:
            df[c] = veh910.loc['i025_age', 'Coefficient'] * NHTS['INC025'] + veh910.loc['i2550_age', 'Coefficient'] * NHTS['INC2550']+ veh910.loc['i100_age', 'Coefficient'] * NHTS['INC100150']+ veh910.loc['i150p_age', 'Coefficient']  * NHTS['INC150P']+ veh910.loc['imiss_age', 'Coefficient']  * NHTS['INC_MISS']+ veh910.loc['oneveh_age', 'Coefficient'] * NHTS['ONEVEH']+ veh910.loc['vhgtdr_age', 'Coefficient'] * NHTS['VHGTDR']+ veh910.loc['nchld_age', 'Coefficient']  * NHTS['NUM_CHILD3'] + veh910.loc['dstwkt_age', 'Coefficient'] * NHTS['DSTWKT']+ veh910.loc['sfo_age', 'Coefficient'] * NHTS['SFO']+ veh910.loc['san_age', 'Coefficient'] * NHTS['SAN']+ veh910.loc['atl_age', 'Coefficient'] * NHTS['ATL']+ veh910.loc['sea_age', 'Coefficient'] * NHTS['SEA']+ veh910.loc['det_age', 'Coefficient'] * NHTS['DET']+ veh910.loc['msp_age', 'Coefficient'] * NHTS['MSP']+ veh910.loc['dca_age', 'Coefficient'] * NHTS['DCA']+ veh910.loc['oreg_age', 'Coefficient'] * NHTS['OREG']+ veh910.loc['ohio_age', 'Coefficient'] * NHTS['OHIO']+ veh910.loc['rural_age', 'Coefficient'] * NHTS['RURAL']+ veh910.loc['cmpg', 'Coefficient'] * 24.0 + veh910.loc['cprice0', 'Coefficient']* NHTS['INC025']* 32927+ veh910.loc['cprice25', 'Coefficient'] * NHTS['INC2550'] * 32927 + veh910.loc['cprice50', 'Coefficient'] * NHTS['INC50100'] * 32927 + veh910.loc['cprice100', 'Coefficient'] * NHTS['INC100150'] * 32927 + veh910.loc['cprice150', 'Coefficient'] * NHTS['INC150P'] * 32927+ veh910.loc['cpricem', 'Coefficient'] * NHTS['INC_MISS'] * 32927
            for d in EPA['diesel']:
                if d > 0:
                    df[d] = df[c] + veh910.loc['dsl', 'Coefficient']
            for h in EPA['hybrid']:
                if h > 0:
                    df[h] = df[c] + veh910.loc['hyb', 'Coefficient']
            for p in EPA['pev']:
                if p > 0:
                    df[p] = df[c] + veh910.loc['pev', 'Coefficient']
            for b in EPA['bev']:
                if b > 0:
                    df[b] = df[c] + veh910.loc['bev', 'Coefficient']
df

In [ ]:
#manual calculation
car = pd.DataFrame()
for j in ages:
    for i in range (1101, 1121):
        car[i] = + veh910.loc['i025_age', 'Coefficient'] * NHTS['INC025']+ veh910.loc['i2550_age', 'Coefficient'] * NHTS['INC2550']+ veh910.loc['i100_age', 'Coefficient'] * NHTS['INC100150']+ veh910.loc['i150p_age', 'Coefficient']  * NHTS['INC150P']+ veh910.loc['imiss_age', 'Coefficient']  * NHTS['INC_MISS']+ veh910.loc['oneveh_age', 'Coefficient'] * NHTS['ONEVEH']+ veh910.loc['vhgtdr_age', 'Coefficient'] * NHTS['VHGTDR']+ veh910.loc['nchld_age', 'Coefficient']  * NHTS['NUM_CHILD3'] + veh910.loc['dstwkt_age', 'Coefficient'] * NHTS['DSTWKT']+ veh910.loc['sfo_age', 'Coefficient'] * NHTS['SFO']+ veh910.loc['san_age', 'Coefficient'] * NHTS['SAN']+ veh910.loc['atl_age', 'Coefficient'] * NHTS['ATL']+ veh910.loc['sea_age', 'Coefficient'] * NHTS['SEA']+ veh910.loc['det_age', 'Coefficient'] * NHTS['DET']+ veh910.loc['msp_age', 'Coefficient'] * NHTS['MSP']+ veh910.loc['dca_age', 'Coefficient'] * NHTS['DCA']+ veh910.loc['oreg_age', 'Coefficient'] * NHTS['OREG']+ veh910.loc['ohio_age', 'Coefficient'] * NHTS['OHIO']+ veh910.loc['rural_age', 'Coefficient'] * NHTS['RURAL']+ veh910.loc['cmpg', 'Coefficient'] * 24.0 + veh910.loc['cprice0', 'Coefficient']* NHTS['INC025']* 32927+ veh910.loc['cprice25', 'Coefficient'] * NHTS['INC2550'] * 32927 + veh910.loc['cprice50', 'Coefficient'] * NHTS['INC50100'] * 32927 + veh910.loc['cprice100', 'Coefficient'] * NHTS['INC100150'] * 32927 + veh910.loc['cprice150', 'Coefficient'] * NHTS['INC150P'] * 32927+ veh910.loc['cpricem', 'Coefficient'] * NHTS['INC_MISS'] * 32927
    for i in range (1201, 1221):
    car[i] = veh910.loc['dsl', 'Coefficient'] + veh910.loc['i025_age', 'Coefficient'] * NHTS['INC025']+ veh910.loc['i2550_age', 'Coefficient'] * NHTS['INC2550']+ veh910.loc['i100_age', 'Coefficient'] * NHTS['INC100150']+ veh910.loc['i150p_age', 'Coefficient']  * NHTS['INC150P']+ veh910.loc['imiss_age', 'Coefficient']  * NHTS['INC_MISS']+ veh910.loc['oneveh_age', 'Coefficient'] * NHTS['ONEVEH']+ veh910.loc['vhgtdr_age', 'Coefficient'] * NHTS['VHGTDR']+ veh910.loc['nchld_age', 'Coefficient']  * NHTS['NUM_CHILD3'] + veh910.loc['dstwkt_age', 'Coefficient'] * NHTS['DSTWKT']+ veh910.loc['sfo_age', 'Coefficient'] * NHTS['SFO']+ veh910.loc['san_age', 'Coefficient'] * NHTS['SAN']+ veh910.loc['atl_age', 'Coefficient'] * NHTS['ATL']+ veh910.loc['sea_age', 'Coefficient'] * NHTS['SEA']+ veh910.loc['det_age', 'Coefficient'] * NHTS['DET']+ veh910.loc['msp_age', 'Coefficient'] * NHTS['MSP']+ veh910.loc['dca_age', 'Coefficient'] * NHTS['DCA']+ veh910.loc['oreg_age', 'Coefficient'] * NHTS['OREG']+ veh910.loc['ohio_age', 'Coefficient'] * NHTS['OHIO']+ veh910.loc['rural_age', 'Coefficient'] * NHTS['RURAL']+ veh910.loc['cmpg', 'Coefficient'] * 24.0 + veh910.loc['cprice0', 'Coefficient']* NHTS['INC025']* 32927+ veh910.loc['cprice25', 'Coefficient'] * NHTS['INC2550'] * 32927 + veh910.loc['cprice50', 'Coefficient'] * NHTS['INC50100'] * 32927 + veh910.loc['cprice100', 'Coefficient'] * NHTS['INC100150'] * 32927 + veh910.loc['cprice150', 'Coefficient'] * NHTS['INC150P'] * 32927 + veh910.loc['cpricem', 'Coefficient'] * NHTS['INC_MISS'] * 32927
car

In [ ]:
#manual calc
car_dis = pd.DataFrame()
for i in range (1201, 1221):
    for j in ages:
        car_dis[i] = j + veh910.loc['dsl', 'Coefficient'] + veh910.loc['i025_age', 'Coefficient'] * NHTS['INC025']+ veh910.loc['i2550_age', 'Coefficient'] * NHTS['INC2550']+ veh910.loc['i100_age', 'Coefficient'] * NHTS['INC100150']+ veh910.loc['i150p_age', 'Coefficient']  * NHTS['INC150P']+ veh910.loc['imiss_age', 'Coefficient']  * NHTS['INC_MISS']+ veh910.loc['oneveh_age', 'Coefficient'] * NHTS['ONEVEH']+ veh910.loc['vhgtdr_age', 'Coefficient'] * NHTS['VHGTDR']+ veh910.loc['nchld_age', 'Coefficient']  * NHTS['NUM_CHILD3'] + veh910.loc['dstwkt_age', 'Coefficient'] * NHTS['DSTWKT']+ veh910.loc['sfo_age', 'Coefficient'] * NHTS['SFO']+ veh910.loc['san_age', 'Coefficient'] * NHTS['SAN']+ veh910.loc['atl_age', 'Coefficient'] * NHTS['ATL']+ veh910.loc['sea_age', 'Coefficient'] * NHTS['SEA']+ veh910.loc['det_age', 'Coefficient'] * NHTS['DET']+ veh910.loc['msp_age', 'Coefficient'] * NHTS['MSP']+ veh910.loc['dca_age', 'Coefficient'] * NHTS['DCA']+ veh910.loc['oreg_age', 'Coefficient'] * NHTS['OREG']+ veh910.loc['ohio_age', 'Coefficient'] * NHTS['OHIO']+ veh910.loc['rural_age', 'Coefficient'] * NHTS['RURAL']+ veh910.loc['cmpg', 'Coefficient'] * 24.0 + veh910.loc['cprice0', 'Coefficient']* NHTS['INC025']* 32927+ veh910.loc['cprice25', 'Coefficient'] * NHTS['INC2550'] * 32927 + veh910.loc['cprice50', 'Coefficient'] * NHTS['INC50100'] * 32927 + veh910.loc['cprice100', 'Coefficient'] * NHTS['INC100150'] * 32927 + veh910.loc['cprice150', 'Coefficient'] * NHTS['INC150P'] * 32927+ veh910.loc['cpricem', 'Coefficient'] * NHTS['INC_MISS'] * 32927
car_dis        

In [ ]:
#manual calc
car_hyb = pd.DataFrame()
for i in range (1301, 1321):
    car_hyb[i] = veh910.loc['hyb', 'Coefficient'] + veh910.loc['vhgtdr_hyb', 'Coefficient'] * NHTS['VHGTDR']+ veh910.loc['den3_hyb', "Coefficient"] * NHTS['DEN3'] + veh910.loc['den4_hyb', 'Coefficient'] * NHTS['DEN4'] + veh910.loc['den5_hyb', 'Coefficient'] * NHTS['DEN5'] + veh910.loc['den6_hyb', 'Coefficient'] * NHTS['DEN6'] + veh910.loc['den7_hyb', 'Coefficient'] * NHTS['DEN7'] + veh910.loc['den8_hyb', 'Coefficient'] * NHTS['DEN8']+ veh910.loc['sfo_mc', "Coefficient"] * NHTS['SFO'] + veh910.loc['san_mc', "Coefficient"] * NHTS['SAN'] + veh910.loc['atl_mc', "Coefficient"] * NHTS['ATL'] + veh910.loc['sea_mc', "Coefficient"] * NHTS['SEA'] + veh910.loc['det_mc', "Coefficient"] * NHTS['DET'] + veh910.loc['msp_mc', "Coefficient"] * NHTS['MSP'] + veh910.loc['dca_mc', "Coefficient"] * NHTS['DCA'] + veh910.loc['oreg_hyb', "Coefficient"] * NHTS['OREG'] + veh910.loc['ohio_hyb', "Coefficient"] * NHTS['OHIO'] + veh910.loc['i025_age', 'Coefficient'] * NHTS['INC025']+ veh910.loc['i025_age', 'Coefficient'] * NHTS['INC025']+ veh910.loc['i2550_age', 'Coefficient'] * NHTS['INC2550']+ veh910.loc['i100_age', 'Coefficient'] * NHTS['INC100150']+ veh910.loc['i150p_age', 'Coefficient']  * NHTS['INC150P']+ veh910.loc['imiss_age', 'Coefficient']  * NHTS['INC_MISS']+ veh910.loc['oneveh_age', 'Coefficient'] * NHTS['ONEVEH']+ veh910.loc['vhgtdr_age', 'Coefficient'] * NHTS['VHGTDR']+ veh910.loc['nchld_age', 'Coefficient']  * NHTS['NUM_CHILD3'] + veh910.loc['dstwkt_age', 'Coefficient'] * NHTS['DSTWKT']+ veh910.loc['sfo_age', 'Coefficient'] * NHTS['SFO']+ veh910.loc['san_age', 'Coefficient'] * NHTS['SAN']+ veh910.loc['atl_age', 'Coefficient'] * NHTS['ATL']+ veh910.loc['sea_age', 'Coefficient'] * NHTS['SEA']+ veh910.loc['det_age', 'Coefficient'] * NHTS['DET']+ veh910.loc['msp_age', 'Coefficient'] * NHTS['MSP']+ veh910.loc['dca_age', 'Coefficient'] * NHTS['DCA']+ veh910.loc['oreg_age', 'Coefficient'] * NHTS['OREG']+ veh910.loc['ohio_age', 'Coefficient'] * NHTS['OHIO']+ veh910.loc['rural_age', 'Coefficient'] * NHTS['RURAL']+ veh910.loc['cmpg', 'Coefficient'] * 24.0 + veh910.loc['cprice0', 'Coefficient']* NHTS['INC025']* 32927+ veh910.loc['cprice25', 'Coefficient'] * NHTS['INC2550'] * 32927 + veh910.loc['cprice50', 'Coefficient'] * NHTS['INC50100'] * 32927 + veh910.loc['cprice100', 'Coefficient'] * NHTS['INC100150'] * 32927 + veh910.loc['cprice150', 'Coefficient'] * NHTS['INC150P'] * 32927+ veh910.loc['cpricem', 'Coefficient'] * NHTS['INC_MISS'] * 32927
for j in ages:
    car_hyb[i] = car_dis[i] + float(j)
car_hyb